## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-08-11-30-07 +0000,wapo,"UPS, FedEx ground MD-11 planes after Louisvill...",https://www.washingtonpost.com/nation/2025/11/...
1,2025-11-08-11-05-12 +0000,nypost,JetBlue passengers accused of having sex in fr...,https://nypost.com/2025/11/08/us-news/jetblue-...
2,2025-11-08-11-01-00 +0000,latimes,Granderson: Is it really an election if there'...,https://www.latimes.com/opinion/story/2025-11-...
3,2025-11-08-11-00-06 +0000,nyt,A Powerful Tool to Override Constitutional Rig...,https://www.nytimes.com/2025/11/08/world/canad...
4,2025-11-08-11-00-06 +0000,wapo,A mom and a pastor shot two fugitives. They we...,https://www.washingtonpost.com/nation/2025/11/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,49
245,up,14
197,shutdown,14
33,court,12
15,flight,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
90,2025-11-08-02-02-29 +0000,nypost,Supreme Court allows Trump admin to temporaril...,https://nypost.com/2025/11/07/us-news/trump-ad...,117
307,2025-11-07-12-40-35 +0000,cbc,Trump administration fights court ruling to pa...,https://www.cbc.ca/news/world/us-ri-court-ruli...,109
80,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...,107
82,2025-11-08-02-59-00 +0000,wsj,Supreme Court Pauses Order for Trump Administr...,https://www.wsj.com/us-news/law/supreme-court-...,106
92,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
90,117,2025-11-08-02-02-29 +0000,nypost,Supreme Court allows Trump admin to temporaril...,https://nypost.com/2025/11/07/us-news/trump-ad...
92,56,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...
115,53,2025-11-07-23-56-29 +0000,nypost,Tesla rival Rivian gives CEO RJ Scaringe a Mus...,https://nypost.com/2025/11/07/business/rivian-...
0,41,2025-11-08-11-30-07 +0000,wapo,"UPS, FedEx ground MD-11 planes after Louisvill...",https://www.washingtonpost.com/nation/2025/11/...
129,41,2025-11-07-22-54-46 +0000,nypost,Top NYC realtor fired for comparing Zohran Mam...,https://nypost.com/2025/11/07/business/top-nyc...
288,39,2025-11-07-14-11-00 +0000,wsj,Senate Majority Leader John Thune expects Sena...,https://www.wsj.com/politics/policy/government...
260,36,2025-11-07-16-04-00 +0000,wsj,Heritage Foundation president Kevin Roberts de...,https://www.wsj.com/politics/policy/the-crack-...
130,35,2025-11-07-22-53-38 +0000,nyt,Flight Disruptions From FAA Cuts Are Limited S...,https://www.nytimes.com/2025/11/07/us/passenge...
112,35,2025-11-08-00-06-04 +0000,nypost,Michigan woman captured hurling piping hot cof...,https://nypost.com/2025/11/07/us-news/michigan...
212,32,2025-11-07-19-00-43 +0000,nyt,Russian Jailed for Placing Tiny Antiwar Signs ...,https://www.nytimes.com/2025/11/07/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
